# **MDP를 모를 때 밸류 평가하기**

지금까지 MDP에서 보상함수 $r^a_s$와 상태 전이확률 $P^a_{ss'}$를 알 때를 기준으로 최적의 정책을 탐색하였음

이제 $r^a_s$와 $P^a_{ss'}$를 모름. 즉, 실제 action을 하기 전에는 특정 상태나 행동을 했을 때 어떤 보상을 얻는지, 어떤 상태로 이동하게 되는지 확률 분포 자체를 모르는 상황 -> Model-Free

## **1. 몬테카를로 학습**

### **1) 개요**
무게가 한 쪽으로 치우친 동전이 있다.( = 사전확률을 0.5로 가정할 수 없다.)

이 동전을 던져서 앞면이 나오면 내가 가질 수 있고 뒷면이 나오면 가질 수 없다.

사전확률이 0.5가 아닌 상황에서 이 동전을 던지는 행위에 대한 기댓값을 구하는 방법은? -> 여러 번 시도한다. -> 이미 존재하지만 우리는 모르는 미지의 확률 분포를 찾아간다. -> **몬테카를로 방법론**

$$ v_\pi(s_t) = E_\pi[G_t] $$

특정 상태의 상태가치함수는 리턴의 기댓값인데, 특정 상태에서의 시행을 반복할 경우 시행 결과를 평균내면 특정 상태의 상태가치함수를 추정할 수 있다.


e.g. 동전 던지기를 10번 해서 앞면이 3회 나왔을 경우 앞면이 나올 확률을 0.3으로 생각할 수 있고(리턴의 기댓값은 앞 3회 * 100원 / 10회 = 30원), 이 시행이 계속 무한히 반복되면 대수의 법칙에 의해 원래 그 동전이 가지고 있는 확률분포의 기댓값에 근사하게 된다.

### **2) 몬테카를로 학습 알고리즘(그리드 월드로 설명 108p ~ 111p)**

**보상**: ? / **상태전이확률**: ? / **정책**: 4방향, uniform random

앞 chapter에서는 $s_2$에서 $a_{right}$ 액션을 선택했다고 가정했을 때, 실제 시행이 없어도 $s_2$에서 $s_3$으로 무조건 이동할 것이라는 것을 알고 있음

왜냐? $P^{a_{right}}_{s2s3} = 1 $이라는 것을 가정했기 때문  

그래서 지난 챕터에서는 에이전트는 실제로 액션을 취하지 않았고 $\pi(a|s)$에다가 $r^a_s+v(s')$를 곱하는 식으로 기존의 상태를 계산하였는데, 이는 planning이라 할 수 있음.

이제 MDP를 모르기 때문에 실제로 액션을 해보지 않으면 우리는 가치를 평가할 수 없음

#### **(1) 테이블 초기화**

4*4(s0 ~ 종료(s15))로 구성된 테이블에 각 상태별로 몇 회 방문했는지$(N(s))$, 해당 상태에서 경험한 리턴의 총합은 얼마인지 $(V(s))$를 기록하기 위해 각 state의 $N(s), V(s)$는 (0,0)으로 초기화

In [1]:
import numpy as np
import pandas as pd
pd.DataFrame([['(0,0)','(0,0)','(0,0)','(0,0)'], 
              ['(0,0)','(0,0)','(0,0)','(0,0)'],
              ['(0,0)','(0,0)','(0,0)','(0,0)'],
             ['(0,0)','(0,0)','(0,0)','(0,0)']])

,0,1,2,3
0,"(0,0)","(0,0)","(0,0)","(0,0)"
1,"(0,0)","(0,0)","(0,0)","(0,0)"
2,"(0,0)","(0,0)","(0,0)","(0,0)"
3,"(0,0)","(0,0)","(0,0)","(0,0)"


#### **(2) 경험 쌓기**
그리드 출발점 $s_0$에서 어떤 액션 $a_0$를 정하고 $s_1$에 도착한 후 그에 따른 보상 $r_0$를 받는다.

이 과정을 반복하면 에이전트는 $s_T$에 도착하는데, 이 에피소드의 시작과 끝은 아래와 같이 나타낼 수 있음

$$ s_0, a_0, r_0, s_1, a_1, r_1, s_2, a_2, r_2 ...  s_{T-1}, a_{T-1}, r_{T-1}, s_T$$

#### **(3) 테이블 업데이트**
지나온 경로가 $s_0 → s_4 → s_5 → s_6 → s_{10} → s_{14} →$종료였을 때, 거쳐온 상태에 대한 $N(s)$, $V(s)$를 알기때문에 초기화했던 테이블을 아래와 같이 업데이트할 수 있음

$$ N(s_t) ← N(s_t) + 1 $$

$$ V(s_t) ← V(s_t) + G_t $$


모든 상태의 보상을 -1, 감마(discount facor)를 1로 계산했을 때,(**단, 여기서의 보상은 에이전트는 모름. 환경에 이미 주어져있다는 의미이며 에이전트가 점차 알게되는 값**)

(1, -6), (1, -5), (1, -4), (1, -3), (1, -2), (1, -1), 종료로 업데이트 가능함 (*지나온 경로별로 1회씩 상태를 경험했기 때문에 $N(s)$는 모두 1)

In [11]:
pd.DataFrame([['s0','s1','s2','s3'], 
              ['s4','s5','s6','s7'],
              ['s8','s9','s10','s11'],
             ['s12','s13','s14','종료']])

,0,1,2,3
0,s0,s1,s2,s3
1,s4,s5,s6,s7
2,s8,s9,s10,s11
3,s12,s13,s14,종료


In [13]:
pd.DataFrame([['(1, -6)','s1','s2','s3'], 
              ['(1, -5)','(1, -4)','(1, -3)','s7'],
              ['s8','s9','(1, -2)','s11'],
             ['s12','s13','(1, -1)','종료']])

,0,1,2,3
0,"(1, -6)",s1,s2,s3
1,"(1, -5)","(1, -4)","(1, -3)",s7
2,s8,s9,"(1, -2)",s11
3,s12,s13,"(1, -1)",종료


#### **(4) 밸류 계산**
경험쌓기, 테이블 업데이트 과정을 충분히 반복한다. 

예를 들어 10만 번 반복한다면, 10만 개의 에피소드에 대한 결과를 얻었을 것이고, 충분한 경험을 쌓았다고 판단되면

최종적으로 상태가치함수에 대한 근사를 하기 위해 모든 상태 s에 대한 리턴의 합 $V(s_t)$를 방문횟수 $N(S)$로 나누면 된다.(= 평균내라는 말, 대문자 V임)

$$ v_\pi(s_t) \cong \frac{V(s_t)}{N(s_t)} $$

### **3) 조금씩 업데이트 하는 버전**
앞 버전과의 차이: 에피소드가 1개 끝날 때마다 테이블 값을 **조금씩** 업데이트(앞 버전은 10만 번의 에피소드가 끝난 후 업데이트)

$$ V(s_t) ← (1-\alpha) * V(s_t) + \alpha * G_t $$


$ \alpha $를 0.1로 가정했을 때, 원래 관측한 $V(s_t)$의 0.9만큼, 그리고 새로 관측한 리턴 $G_t$에 대해 0.1만큼만 업데이트 하는 것

-> $\alpha$의 값에 따라 큰 폭, 작은 폭으로 업데이트가 이루어지며, 위 방식과는 달리 $N(s_t)$를 저장할 필요 없이, 매 에피소드가 끝날 때마다 테이블을 업데이트 할 수 있음.    

위 식은 
$$ = V(s_t) ← V(s_t) + \alpha(G_t - V(s_t)) $$  
이렇게 나타낼 수 있는데, $G_t$가 $V(s_t)$보다 크면 $\alpha$에 곱해진 값이 양수가 되어 기존의 $V(s_t)$ 값을 더 크게 만든다, 반대의 경우에는 $V(s_t)$를 더 작게 만든다(얼마만큼 
커지고 작아지는지 직관적으로 이해할 수 있음)

### **4) 몬테카를로 학습 과정**

학습을 위해 필요한 것

1. 환경: 에이전트의 액션을 받아 상태변이를 일으키고, 보상을 줌
2. 에이전트: 4방향 랜덤 정책을 이용해 액션을 함
3. 경험 쌓는 부분: 에이전트가 환경과 상호작용하며 데이터를 축적
4. 학습하는 부분: 쌓인 경험을 테이블에 업데이트

In [14]:
from ch5_MCLearning import GridWorld, Agent

## GridWorld 클래스

In [2]:
import random
import numpy as np

class GridWorld():
    
    # 생성자. 초깃값 설정
    def __init__(self):
        self.x=0
        self.y=0
        
    # 액션에 대한 상태 변이, 보상, 중단여부를 리턴하는 함수
    def step(self, a):
        # 0번 액션: 왼쪽, 1번 액션: 위, 2번 액션: 오른쪽, 3번 액션: 아래쪽에 대한 상태변이 정의
        if a==0:
            self.move_left()
        elif a==1:
            self.move_up()
        elif a==2:
            self.move_right()
        elif a==3:
            self.move_down()
        # action에 의한 보상은 항상 -1로 고정
        reward = -1 
        done = self.is_done()
        # x, y의 좌표, reward, 중단여부 리턴
        return (self.x, self.y), reward, done
    
    # action의 속성 정의(행과 열이 반대라서 right / left가 y, up / down이 x)
    def move_right(self):
        # 에피소드가 끝나야 하니까 += 연산자 사용
        self.y += 1  
        # 4*4 여서 0~3 범위내에 있어야함(그리드 월드에서는 규정된 범위보다 작거나 크면 안움직이는 것으로 규정함)
        if self.y > 3:
            self.y = 3
      
    def move_left(self):
        self.y -= 1
        if self.y < 0:
            self.y = 0
      
    def move_up(self):
        self.x -= 1
        if self.x < 0:
            self.x = 0
  
    def move_down(self):
        self.x += 1
        if self.x > 3:
            self.x = 3
    # 중단 상태인 (3,3)에 도달하면 is_done = True라서 step에서 done을 리턴하게 해줌
    def is_done(self):
        if self.x == 3 and self.y == 3:
            return True
        else :
            return False
    # 현재 x, y의 좌표 리턴
    def get_state(self):
        return (self.x, self.y)
    # 종료상태 도달했을 때 초기화하기 위한 함수
    def reset(self):
        self.x = 0
        self.y = 0
        return (self.x, self.y)

## Agent 클래스

In [3]:
class Agent():
    # 생성자
    def __init__(self):
        pass
    
    # 액션을 uniform하게 랜덤으로 선택
    def select_action(self):
        coin = random.random()
        if coin < 0.25:
            action = 0
        elif coin < 0.5:
            action = 1
        elif coin < 0.75:
            action = 2
        else:
            action = 3
        return action

## main함수

In [20]:
def main():
    # env, agent 인스턴스
    env = GridWorld()
    agent = Agent()
    # 테이블 초깃값 설정
    data = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
    # 보상, 할인율, 업데이트 파라미터 정의
    gamma = 1.0
    reward = -1
    alpha = 0.001
    
    # 총 5만 번의 에피소드를 진행하는데, 
    for k in range(50000):
        done = False
        history = []
        
        # 에이전트가 경험을 쌓는 과정. not done이 True이면(done이 False인게 True면) 아래 변수들을 만드는데, done = True이면 not done은 False이니까 멈춘다. 
        while not done:
            # agent는 select_action()을 하고
            action = agent.select_action()
            # (x,y)의 좌표, 보상, 중단여부 변수를 만들고(env에서 정의한 step 함수, is_done 함수 등에 의해 (self.x, self.y), reward, done 를 리턴)
            (x,y), reward, done = env.step(action)
            # 그리고 그 값을 history에 append한다.
            history.append((x,y,reward))
        # 에피소드가 한 번 끝나면 환경 리셋
        env.reset()
        
        
        # 매 에피소드가 끝나고 바로 해당 데이터를 이용해 테이블을 업데이트
        cum_reward = 0
        
        # 쌓은 경험을 통해 테이블을 업데이트하는 영역
        # transition이 history 테이블의 역순 정렬(append 된 시점 기준 역순)에 있으면, == 방문했던 상태들을 뒤에서부터 보겠다는 의미
        # G_t와 G_t+1은 재귀적 관계가 있으므로 역순으로 정렬(G_t = R_t+1 + gammaG_t+1)
        for transition in history[::-1]:
            # x, y, reward에 transition의 밸류를 넣고,
            x, y, reward = transition
            # 테이블에서 state별 밸류를 업데이트 한다. V(s_t) <- V(s_t) + alpha(G_t - V(s_t))
            data[x][y] = data[x][y] + alpha*(cum_reward-data[x][y])
            # 각 state의 cum_reward를 업데이트 한다.
            cum_reward = cum_reward + gamma*reward
#             print(data[x][y])
            
    print("업데이트 결과: ")           
    for row in data:
        # row 출력 - 굳이 for를 쓰는 건 그대로 print(data)하면 list로 나오니깐 보기 불편함
        # 여기서 row에 있는 값은 data[x][y] = data[x][y] + alpha*(cum_reward-data[x][y])를 통해 업데이트한 값
        print(row)
        
if __name__ == '__main__':
    main()

업데이트 결과: 
[-60.790709554349945, -58.3862365554446, -53.3399901121054, -47.65292391597761]
[-56.528237590692726, -53.80719151893177, -48.02114054484271, -41.73786634594]
[-56.553299207497226, -50.86077709516569, -40.298866020675796, -28.88894104534365]
[-54.53325344736141, -45.9914139601591, -29.829851181883573, 0.0]


In [21]:
def main():
    # env, agent 인스턴스
    env = GridWorld()
    agent = Agent()
    # 테이블 초깃값 설정
    data = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
    # 보상, 할인율, 업데이트 파라미터 정의
    gamma = 1.0
    reward = -1
    alpha = 0.001
    
    # 총 5만 번의 에피소드를 진행하는데, 
    for k in range(3):
        done = False
        history = []
        
        # 에이전트가 경험을 쌓는 과정. not done이 True이면(done이 False인게 True면) 아래 변수들을 만드는데, done = True이면 not done은 False이니까 멈춘다. 
        while not done:
            # agent는 select_action()을 하고
            action = agent.select_action()
            # (x,y)의 좌표, 보상, 중단여부 변수를 만들고(env에서 정의한 step 함수, is_done 함수 등에 의해 (self.x, self.y), reward, done 를 리턴)
            (x,y), reward, done = env.step(action)
            # 그리고 그 값을 history에 append한다.
            history.append((x,y,reward))
        print("history: " + str(history))
        # 에피소드가 한 번 끝나면 환경 리셋
        env.reset()
        
        
        # 매 에피소드가 끝나고 바로 해당 데이터를 이용해 테이블을 업데이트
        cum_reward = 0
        
        # 쌓은 경험을 통해 테이블을 업데이트하는 영역
        # transition이 history 테이블의 역순 정렬(append 된 시점 기준 역순)에 있으면, == 방문했던 상태들을 뒤에서부터 보겠다는 의미
        # G_t와 G_t+1은 재귀적 관계가 있으므로 역순으로 정렬(G_t = R_t+1 + gammaG_t+1)

        for transition in history[::-1]:
            # x, y, reward에 transition의 밸류를 넣고,
            x, y, reward = transition
            # 테이블에서 state별 밸류를 업데이트 한다. V(s_t) <- V(s_t) + alpha(G_t - V(s_t))
            data[x][y] = data[x][y] + alpha*(cum_reward-data[x][y])
            # 각 state의 cum_reward를 업데이트 한다.
            cum_reward = cum_reward + gamma*reward
            print("state({}, {})의 누적 보상: ".format(x, y) +str(cum_reward))
            
        print("\n")
            
    print("업데이트 결과: ")           
    for row in data:
        # row 출력 - 굳이 for를 쓰는 건 그대로 print(data)하면 list로 나오니깐 보기 불편함
        # 여기서 row에 있는 값은 data[x][y] = data[x][y] + alpha*(cum_reward-data[x][y])를 통해 업데이트한 값
        print(row)

if __name__ == '__main__':
    main()

history: [(1, 0, -1), (1, 1, -1), (2, 1, -1), (1, 1, -1), (2, 1, -1), (2, 2, -1), (3, 2, -1), (3, 3, -1)]
state(3, 3)의 누적 보상: -1.0
state(3, 2)의 누적 보상: -2.0
state(2, 2)의 누적 보상: -3.0
state(2, 1)의 누적 보상: -4.0
state(1, 1)의 누적 보상: -5.0
state(2, 1)의 누적 보상: -6.0
state(1, 1)의 누적 보상: -7.0
state(1, 0)의 누적 보상: -8.0


history: [(1, 0, -1), (0, 0, -1), (0, 1, -1), (0, 0, -1), (0, 0, -1), (0, 1, -1), (1, 1, -1), (2, 1, -1), (1, 1, -1), (2, 1, -1), (3, 1, -1), (3, 2, -1), (3, 1, -1), (3, 1, -1), (3, 0, -1), (3, 1, -1), (3, 2, -1), (3, 2, -1), (3, 2, -1), (2, 2, -1), (1, 2, -1), (2, 2, -1), (1, 2, -1), (0, 2, -1), (0, 1, -1), (0, 0, -1), (0, 0, -1), (0, 0, -1), (1, 0, -1), (1, 0, -1), (1, 1, -1), (2, 1, -1), (2, 2, -1), (3, 2, -1), (3, 3, -1)]
state(3, 3)의 누적 보상: -1.0
state(3, 2)의 누적 보상: -2.0
state(2, 2)의 누적 보상: -3.0
state(2, 1)의 누적 보상: -4.0
state(1, 1)의 누적 보상: -5.0
state(1, 0)의 누적 보상: -6.0
state(1, 0)의 누적 보상: -7.0
state(0, 0)의 누적 보상: -8.0
state(0, 0)의 누적 보상: -9.0
state(0, 0)의 누적 보상: -10.0
state(0, 1)

In [66]:
t = [[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]]
# t[1][1]
for row in t:
    print(row)

[1, 2, 3, 4]
[5, 6, 7, 8]
[9, 10, 11, 12]
[13, 14, 15, 16]


In [15]:
t = [1, 2, 3, 4, 5]
t[::-1]

[5, 4, 3, 2, 1]